In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../utils'))

import numpy as np
import file_utils
from itertools import chain, combinations

# Day 11: Seating System
Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!

By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).

The seat layout fits neatly on a grid. Each position is either floor (.), an empty seat (L), or an occupied seat (#). For example, the initial seat layout might look like this:

```
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
```

Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the number of occupied seats adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are applied to every seat simultaneously:

If a seat is empty (L) and there are no occupied seats adjacent to it, the seat becomes occupied.
If a seat is occupied (#) and four or more seats adjacent to it are also occupied, the seat becomes empty.
Otherwise, the seat's state does not change.
Floor (.) never changes; seats don't move, and nobody sits on the floor.

After one round of these rules, every seat in the example layout becomes occupied:

```
#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
```

After a second round, the seats with four or more occupied adjacent seats become empty again:

```
#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##
```

This process continues for three more rounds:

```
#.##.L#.##
#L###LL.L#
L.#.#..#..
#L##.##.L#
#.##.LL.LL
#.###L#.##
..#.#.....
#L######L#
#.LL###L.L
#.#L###.##
```
```
#.#L.L#.##
#LLL#LL.L#
L.L.L..#..
#LLL.##.L#
#.LL.LL.LL
#.LL#L#.##
..L.L.....
#L#LLLL#L#
#.LLLLLL.L
#.#L#L#.##
```
```
#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##
```

At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state! Once people stop moving around, you count 37 occupied seats.

Simulate your seating area by applying the seating rules repeatedly until no seats change state. How many seats end up occupied?

In [2]:
seats = file_utils.read(2020, 11)
seats = np.array([list(row) for row in seats])

print(seats)

[['L' 'L' 'L' ... 'L' 'L' 'L']
 ['L' 'L' 'L' ... 'L' 'L' 'L']
 ['L' 'L' 'L' ... 'L' 'L' 'L']
 ...
 ['L' 'L' 'L' ... 'L' 'L' 'L']
 ['L' 'L' 'L' ... 'L' 'L' 'L']
 ['L' 'L' 'L' ... 'L' 'L' 'L']]


In [3]:
def adjacent_seats(seats, x, y):
    adjacent = []

    if y > 0:
        # check upper
        adjacent.append(seats[y-1,x])    

    if y > 0 and x < seats.shape[1] - 1:
        # check upper right
        adjacent.append(seats[y-1,x+1])

    if x < seats.shape[1] - 1:
        # check right
        adjacent.append(seats[y,x+1])

    if y < seats.shape[0] - 1 and x < seats.shape[1] - 1:
        # check down right 
        adjacent.append(seats[y+1,x+1])

    if y < seats.shape[0] - 1:
        # check down
        adjacent.append(seats[y+1,x])

    if y < seats.shape[0] - 1 and x > 0:
        # check down left
        adjacent.append(seats[y+1,x-1])

    if x > 0:
        # check left  
        adjacent.append(seats[y,x-1])

    if y > 0 and x > 0:
        # check upper left
        adjacent.append(seats[y-1,x-1])

    return np.array(adjacent)

In [4]:
def simulate_seating(seats):
    updated = seats.copy()
    for y, row in enumerate(seats):
        for x, seat in enumerate(row):
            if seat_is_ready_to_be_occupied(seats, x, y):
                updated[y,x] = '#'
            if seat_is_left(seats, x, y):
                updated[y,x] = 'L'
    
    if (seats == updated).all():
        return updated, True # stabilized
    return updated, False

def seat_is_ready_to_be_occupied(seats, x, y):
    seat = seats[y, x]
    if seat == 'L':
        s = adjacent_seats(seats, x, y)
        return np.sum(s == '#') == 0
    return False

def seat_is_left(seats, x, y):
    seat = seats[y, x]
    if seat == '#':
        s = adjacent_seats(seats, x, y)
        return np.sum(s == '#') >= 4
    return False

In [5]:
stabilized = False
s = seats
while not stabilized:
    s, stabilized = simulate_seating(s)

len(np.where(s == '#')[0])

2194

# Part Two
As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about the first seat they can see in each of those eight directions!

Now, instead of considering just the eight immediately adjacent seats, consider the first seat in each of those eight directions. For example, the empty seat below would see eight occupied seats:

```
.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
```

The leftmost empty seat below would only see one empty seat, but cannot see any of the occupied ones:

```
.............
.L.L.#.#.#.#.
.............
```

The empty seat below would see no occupied seats:

```
.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
```

Also, people seem to be more tolerant than you expected: it now takes five or more visible occupied seats for an occupied seat to become empty (rather than four or more from the previous rules). The other rules still apply: empty seats that see no occupied seats become occupied, seats matching no rule don't change, and floor never changes.

Again, at this point, people stop shifting around and the seating area reaches equilibrium. Once this occurs, you count 26 occupied seats.

Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, how many seats end up occupied?

In [6]:
def within_bounds(seats, x, y):
    return x < seats.shape[1] and x >= 0 and y < seats.shape[0] and y >= 0

def is_taken(seats, x, y):
    return seats[y,x] in ['#', 'L']

def find_visible_seats(seats, x, y):
    directions = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]
    line_of_sight = []

    for d in directions:
        coordinates = (x, y)
                
        while True:   
            coordinates = (coordinates[0] + d[0], coordinates[1] + d[1])
            if within_bounds(seats, *coordinates):
                if is_taken(seats, *coordinates):
                    line_of_sight.append(seats[coordinates[1], coordinates[0]])
                    break
            else:
                break

    return np.array(line_of_sight)

find_visible_seats(seats, 4, 4)

array(['L', 'L', 'L', 'L', 'L', 'L', 'L', 'L'], dtype='<U1')

In [7]:
def seat_is_left_with_los(seats, x, y):
    seat = seats[y, x]
    if seat == '#':
        s = find_visible_seats(seats, x, y)
        return np.sum(s == '#') >= 5
    return False

def seat_is_ready_to_be_occupied_with_los(seats, x, y):
    seat = seats[y, x]
    if seat == 'L':
        s = find_visible_seats(seats, x, y)
        return np.sum(s == '#') == 0
    return False

def simulate_seating_with_los(seats):
    updated = seats.copy()
    for y, row in enumerate(seats):
        for x, seat in enumerate(row):
            if seat_is_ready_to_be_occupied_with_los(seats, x, y):
                updated[y,x] = '#'
            if seat_is_left_with_los(seats, x, y):
                updated[y,x] = 'L'
    
    if (seats == updated).all():
        return updated, True # stabilized
    return updated, False

In [8]:
stabilized = False
s = seats
while not stabilized:
    s, stabilized = simulate_seating_with_los(s)

len(np.where(s == '#')[0])

1944